In [ ]:
import re
import math
import pandas as pd
import numpy as np

from scipy.stats import ttest_1samp, ttest_ind


# 베이지안

In [ ]:
# 사전확률
A = 1/2
not_A = 1/2

strawberry_for_A = 7/10
strawberry_for_B = 5/10

# 사후확률
posterior = (strawberry_for_A * A) / ((strawberry_for_A * A) + (strawberry_for_B * not_A))

A, round(posterior, 2)

# 부트스트래핑

In [ ]:
# 200 samples bootstrapping
sample_200 = []

for _ in range(10000):
  sample200 = np.random.choice(final['Income'], 200, replace=True)
  sample_200.append(sample200.mean())

# 신뢰구간 95% 



lower, upper = np.percentile(sample_200, 2.5), np.percentile(sample_200, 97.5)

round(lower, 2), round(upper, 2)

# T 검정

    T-test : 두 집단의 평균 차이 검증
        원리상 표본이 충분이 크면 대부분 유의미로 나온다.

> 고객의 자녀가 청소년인 경우의 연간 수입 평균값이 그렇지 않은 경우보다 더 많은지 가설 검증을 시행하고자 합니다.
 
- 자녀가 청소년인 경우 고객의 연간 수입 평균값: $μ_{yes}$
- 자녀가 청소년이 아닌 경우 고객의 연간 수입 평균값: $μ_{no}$ 

#### One-Sample T-test (단측검정)

In [ ]:
# 가설 검정 T-test case 01
#  one-tailed test (단측검정)
#
# 부트 스트래핑 과정이 포함 되어 있음
# 

sample # 셈플 값이 들어 있는 배열
pop_mean # 비교 하고자 하는 값  ex) 키 172.5

pv1 = ttest_1samp( sample , pop_mean ).pvalue
if pv1 >= 0.05:
    print(f"P-value = {pv1.round(3)} >= 0.05 이므로")
    print("귀무가설을 채택")
else:
    print(f"P-value = {pv1.round(3)} < 0.05 이므로")
    print("귀무가설을 기각")

In [ ]:
# 가설 검정 T-test case 02
#  one-tailed test (단측검정)
#

# seed는 2로 지정합니다.
np.random.seed(2)

H0 : h_yes - h_no =< 0
Ha : h_yes - h_no > 0

# Teenhome 값에 따른 분류
pop_yes_means, pop_no_means, pop_diff_means = [], [], []

# Bootstrapping
# 셈플에서 셈프의 수만큼 중복 추출하여
# 그중 h0 조건의 평균 값과 , ha 조건의 평균값, 두 평균값의 차이값 을 
# 충분히 모으면 이는 정규분포를 따른다.  ( ex에서는 1만번)  
for _ in range(10000):
  sample = final_sample.sample(final_sample.shape[0], replace=True)

  yes_mean = sample.loc[sample['Teenhome'] == True, 'Income'].mean()
  pop_yes_means.append(yes_mean)

  no_mean = sample.loc[sample['Teenhome'] == False, 'Income'].mean()
  pop_no_means.append(no_mean)

  pop_diff_means.append(yes_mean - no_mean)



np.random.seed(2)

# 귀무분포 구하기
# 귀무가설이 맞다고 가정한 경우의 분포 구하기
#   모수에 대한 귀무분포를 구한다.
#
#   ex) 우리가 얻은 데이터에 있는 두 표본 집단이 같은 모집단에서부터 나온거라고 치자.
#         그랬을 때, 우리가 이런 검정 통계량(가령, t-value)을 얻었는데 이게 얼마나 말이되는거냐?"
#
#
# 중심(평균값)이 0 이고 동일한 표준편차를 가지는 정규분포를 이용하여 검증한다.
pop_diff_std = np.std(pop_diff_means) # 모수의 표준편차
null_vals = np.random.normal(0, pop_diff_std, 10000) # 중심값이 0인 귀무분포

# 판단 기준이 되는 귀무가설 값 구하기
#  이경우 셈플에서 차이값의 평균 이며 , 특정 비교 값이 정해져 있는 경우에는 그 값으로 비교 하게 될 것이다. 
sample_diff_mean = final_sample.loc[final['Teenhome'] == True, 'Income'].mean() - final_sample.loc[final['Teenhome'] == False, 'Income'].mean()
귀무가설값 = sample_diff_mean

# pvalue 계산
"""
  모집단의 분포에서 95% 신뢰구간을 확인한 뒤, 
  우리의 귀무가설이 이 표본 분포 어디에 해당하는지 확인하여 
  귀무가설을 채택할지 기각할지를 결정한다. 
"""
pvalue = (null_vals > 귀무가설값).mean()

"""
pvalue = 
        1. 귀무가설이 맞다는 전제 하에, 통계값이 실제로 관측된 값 (샘플의 통계치) 이상일 확률을 의미합니다
        2. 우리가 얻은 검정 통계량보다 크거나 같은 값을 얻을 수 있을 확률
pvalue 가 0.05 보다 크면 귀무가설 채택 
pvalue 가 0.05 보다 작으면 대립가설 채택
"""


# 시각화
plt.hist(null_vals, alpha=0.5, label='null hypothesis')
plt.axvline(sample_diff_mean, color='r', label='statistics')
plt.legend(loc=9, fontsize=13)




#### 양측검정

In [ ]:
# T test 프로세스 함수 정의  case 01
# 양측 검정 

from scipy import stats

# 독립집합인 a ,b 에 대한 T-test 수행 결과를 print 하고 P-value를 반환
# 에러시 np.nan 반환
'''
  a : sample 그룹 ( 부트스트래핑이 안되어 있는)
  b : sample 그룹 ( 부트스트래핑이 안되어 있는)

  prm_alternative : 
    'less' : 대안가설이 = 왼쪽의 평균이  오른쪽 보다 작다 인 경우
    'greater' 
    'two-sided' : 대안가설이 = 왼쪽 오른쪽의 평균이 다르다 인 경우  
'''
def t_test ( a, b , prm_alternative ) :
  ret_value = np.nan


  # 정규성 검사 
  # : T-검정은 데이터의 정규본포를 가정으로 하는 검증법이다.
  # a0_pvalue 와 a1_pvalue 모두 > 0.05 보다 큰지 여부 확인 하여 
  # 둘 다 정규성을 만족하지 않는 경우 진행 중단.
  # shapiro test p>0.05이면 정규성이 보장된다. 
  sha_a0, a0_pvalue = stats.shapiro(a)
  sha_a1, a1_pvalue = stats.shapiro(b)
  if ( (a0_pvalue <= 0.05) | (a1_pvalue <= 0.05) ) :
    print (f"(todo) 에러 발생. {a0_pvalue} , {a1_pvalue} ")
    return ret_value


 
  # 분산의 동질성 검정
  # scipy.ttest_ind ( ) 를 사용하기 위해서는 분산 동질성 여부를 알고 있어야 한다.
  flg_분산동질여부 = False
  l, p = stats.levene(a,b)
  if p > 0.05 : # 0.05보다 큰 경우 분산은 동질성을 가지고 있다고 할 수 있음
    flg_분산동질여부 = True


  # T 검정 수행
  # alternative='less' : 대안가설이 = 왼쪽의 평균이  오른쪽 보다 작다 인 경우
  # greater
  # alternative='two-sided' : 대안가설이 = 왼쪽 오른쪽의 평균이 다르다 인 경우  
  #
  # 내부에 부트스트래핑 과정이 포함되어 있음
  t, p = stats.ttest_ind(a, b , equal_var= flg_분산동질여부 , alternative=prm_alternative )


  # 결과출력
  # p < 0.05 인 귀무가설 기각
  rslt_맞는가설 = '귀무가설'
  if ( p < 0.05 ) :
      rslt_맞는가설 = '대립가설'
  print ( f"{rslt_맞는가설}이 맞음 p= {p}" )

  ret_value = p 
  
  return ret_value



# 가설 검정 함수 호출 
np.random.seed(2)

pvalue = t_test ( lst_sample_a , lst_sample_b , "two-sided" )
pvalue


#### **$\chi^2$ 가설 검정*
: 분포의 동일성 검증,  
변수의 연관성 검증 시 사용

- one sample $\chi^2$ - test (적합도 검정)
  > **주어진 하나의 범주형 변수의 분포가 특정 예상되는 분포와 동일한 분포를 나타내는지** 에 대한 가설 검정 방법

  - 귀무가설($H_0$): 주어진 변수 X의 관측분포와 특정 예상 분포가 동일하다.
  - 대립가설($H_a$): 주어진 변수 X의 관측분포와 특정 예상 분포가 다르다.  
  
  
- two sample $\chi^2$ - test (독립성 검정)
  > **두 개의 범주형 변수가 서로 연관되어 있는지 여부**에 대한 가설 검정 방법

  - 귀무가설($H_0$): 변수 X와 Y는 서로 독립이다. (연관 없음)
  - 대립가설($H_a$): 변수 X와 Y는 서로 독립이 아니다. (연관 있음)

In [ ]:
#### Two sample 테스트 